In [1]:
import os
import sys

In [2]:
!pip install -r requirements.txt

     |████████████████████████████████| 60.5 MB 53.5 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 5.8 MB 52.2 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 47.0 MB/s eta 0:00:01
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61086 sha256=8546b8d81ebe321fc85225afbf9279bd7cff888aa4a2c8f61aad70c93d2f87fb
  Stored in directory: /home/jovyan/.cache/pip/wheels/b1/1a/8f/a4c34be976825a2f7948d0fa40907598d69834f8ab5889de11
Successfully built PyPDF2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0


In [3]:
!conda install -c conda-forge poppler -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.0
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.0
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - poppler


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    cairo-1.16.0               |    h9f066cc_1006         1.5 MB  conda-forge
    certifi-2021.10.8          |   py38h578d9bd_1         145 KB  conda-forge
    curl-7.71.1                |       he644dc0

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
import PyPDF2
import os
import tempfile
import zipfile,fnmatch
import csv
import pandas as pd
import numpy as np
import json
import math 
import cv2
import operator
from PIL import Image, ImageDraw
import operator
from skimage.io import imread
from PyPDF2 import PdfFileWriter, PdfFileReader
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
from PIL import Image
import sklearn.metrics

In [6]:
rootPath = r"/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/bleska.ufa.cas.cz/lsbb/storage/elm/2016/12"
rootPath

'/home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/bleska.ufa.cas.cz/lsbb/storage/elm/2016/12'

In [ ]:
pattern = '*.zip'
for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(root, os.path.splitext(filename)[0]))
        os.remove(os.path.join(root, filename))

for root, dirs, files in os.walk(rootPath):
    for file in files:
        if file.endswith(".pdf")==False:
            os.remove(os.path.join(root, file))

In [7]:
# parameters to crop images (in pixels)
left = 154
upper = 1392
right = 1575
lower = 1554

In [ ]:
# crop and save 9 images from one pdf in form "ch2_20150515_220451_199_s37.jpg"
for root, dirs, files in os.walk(rootPath):
    graphMetaData=[]
    for file in files:
        if file.endswith(".pdf"):
            print("####################################")
            print("processing %s" %root)
            graphMetaData=[]
            pdfFileObj = open(os.path.join(root, file), 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
            pageObjDate = pdfReader.getPage(0)
            textDate=pageObjDate.extractText()
            if "from " in textDate:
                dateOfGraph=textDate[textDate.find('from ')+5:textDate.find('from ')+29]
            else: 
                print("pdf from %s is empty, lets continue")
                break
            dateOfGraph=dateOfGraph.replace("-", "")
            dateOfGraph=dateOfGraph.replace("T", "_")
            dateOfGraph=dateOfGraph.replace(".", "_", 1)
            dateOfGraph=dateOfGraph.replace(".", "")
            dateOfGraph=dateOfGraph.replace(":", "")
            for j in range(pdfReader.numPages):
                pageObj = pdfReader.getPage(j)
                text=pageObj.extractText()
                if "Part" in text:
                    graphMetaData.append(text[text.find('Part')+5:text.find('/')])
                else:
                    graphMetaData.append(0)

            with tempfile.TemporaryDirectory() as path:
                images_from_path = convert_from_path(os.path.join(root, file), output_folder=path)

            orderNr=0
            for page in images_from_path:
                page.save(os.path.join(root,"%s. %s.jpg" % (orderNr, graphMetaData[orderNr])), 'JPEG')   
                orderNr+=1

            os.remove(os.path.join(root, "0. 0.jpg"))
            os.remove(os.path.join(root, "%s. %s.jpg" % (orderNr-1, graphMetaData[orderNr-1])))
            graphMetaData.remove(graphMetaData[0])
            graphMetaData.remove(graphMetaData[orderNr-2])
            for item in graphMetaData:
                im = Image.open(os.path.join(root, "%s. %s.jpg" % (str(graphMetaData.index(item)+1), item)))
                im.crop((left, upper, right, lower)).save(os.path.join(root, "ch2_%s_s%s.jpg"%(dateOfGraph, item)))
                os.remove(os.path.join(root, "%s. %s.jpg" % (str(graphMetaData.index(item)+1), item)))
            os.remove(os.path.join(root, "data.pdf"))
        else:
            continue

####################################
processing /home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/bleska.ufa.cas.cz/lsbb/storage/elm/2016/12/01/elm_20161201_000315_pics
####################################
processing /home/jovyan/data/lightning/Samuel/git/tweeks_detection/files/2016_Data/bleska.ufa.cas.cz/lsbb/storage/elm/2016/12/01/elm_20161201_000815_pics
